
# Banking Assistant Chatbot with Learning Mechanism
This notebook demonstrates a simple banking chatbot with:
- Natural Language Interface
- Inference Engine
- Knowledge Base (using SQLite)
- Learning mechanism for unrecognized queries


In [1]:
# Install NLTK (if not installed)
!pip install nltk



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

import nltk
import sqlite3
from nltk.tokenize import word_tokenize

nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\STZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:

# Initialize in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Table for static account types
cursor.execute('''
CREATE TABLE accounts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    account_type TEXT NOT NULL
)
''')
cursor.executemany('INSERT INTO accounts (account_type) VALUES (?)',
                   [('Savings Account',), ('Current Account',), ('Fixed Deposit Account',)])
conn.commit()

# Table for unknown/learning queries
cursor.execute('''
CREATE TABLE IF NOT EXISTS learning (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    question TEXT NOT NULL
)
''')
conn.commit()


In [4]:

# Static Q&A knowledge base
static_responses = {
    'hello': 'Hi there! How can I help you today?',
    'thank you': 'You are welcome!',
    'good morning': 'Good morning! How may I assist you?',
    'help': 'Sure! I can help you with account types, basic queries, and more. Just ask!'
}


In [5]:
def get_response(user_input):
    user_input = user_input.lower().strip()

    # Check full input first for static responses
    if user_input in static_responses:
        return static_responses[user_input]

    tokens = word_tokenize(user_input)

    # Check for known single-word tokens
    for token in tokens:
        if token in static_responses:
            return static_responses[token]

    # Dynamic account query
    if 'account' in tokens or 'accounts' in tokens:
        cursor.execute('SELECT account_type FROM accounts')
        accounts = cursor.fetchall()
        return 'We offer the following accounts: ' + ', '.join([a[0] for a in accounts])

    # Log unknown queries for learning
    cursor.execute('INSERT INTO learning (question) VALUES (?)', (user_input,))
    conn.commit()
    return "I'm sorry, I didn't understand that. Can you please rephrase?"


In [ ]:

# Chat interface
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Bot: Thank you for chatting! Goodbye.")
        break
    response = get_response(user_input)
    print(f"Bot: {response}")


You:  hello


Bot: Hi there! How can I help you today?


You:  good morning


Bot: Good morning! How may I assist you?


You:  help


...

Bot: We offer the following accounts: Savings Account, Current Account, Fixed Deposit Account


You:  thank you


Bot: You are welcome!


You:  tell me about loan


Bot: I'm sorry, I didn't understand that. Can you please rephrase?
